In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# number 1 to 10 data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction,feed_dict={xs:v_xs,keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys,keep_prob:1})
    return result

In [27]:
#define weights
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)


In [28]:
#define CNN Layer
# stride[1,x_movemnet,y_movement, 1]
# padding 2 ways: 'SAME' (抽取的时候边上有0， 抽取完的图片长宽和原图一样)AND 'VALID'（抽取的时候在图片里面， 抽取完的图片比原图小）
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding = 'SAME')
    

In [29]:
#define max pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


In [30]:
#define placeholder for input to network
xs = tf.placeholder(tf.float32,[None,784])
ys = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)
# [-1,28,28,1] -1:all image, 28*28:all pixel ,1:one coloe,gray; if RGB will be 3
x_image = tf.reshape(xs,[-1,28,28,1])

In [31]:
# conv1 layer
#patch 5*5, image 厚度 1， out size 输出高度 32
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_varibale([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)# output size =28*28*32 因为same padding 抽出，长款不变，高度变成32
h_pool1 = max_pool_2x2(h_conv1) #output size = 14*14*32 因为步长为2，所以长宽都缩小2倍

In [32]:
#conv2 layer
#patch 5*5, image 厚度 变成32，因为之前一层的输出高度为32， out size 输出高度 64
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_varibale([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)# output size =14*14*64 因为same padding 抽出，长款不变，高度变成32
h_pool2 = max_pool_2x2(h_conv2) #output size = 7*7*64 因为步长为2，所以长宽都缩小2倍

In [35]:
#func1 layer
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
#[n_samples,7,7,64]-->[n_samples,7*7*64]
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#func2 layer
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
#output use softmax
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)


In [36]:
#the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [42]:
init = tf.global_variables_initializer()

In [47]:


with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys,keep_prob: 0.5})
        if i% 50 == 0:
            print(compute_accuracy(mnist.test.images[:1000],mnist.test.labels[:1000]))
            

0.123
0.905
0.95
0.964
0.967
0.976
0.982
0.982
0.986
0.982
0.985
0.98
0.982
0.983
0.983
0.982
0.987
0.987
0.983
0.985
